<a href="https://colab.research.google.com/github/SergioTD3/PES/blob/main/DETECTOR_DE_ESTACIONAMIENTO_ETAPA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# DETECTOR DE ESTACIONAMIENTO_ETAPA 1




In [1]:
import cv2
import numpy as np
import pickle
import os
import math
import time


# ---------------------------
# RUTAS
# ---------------------------

VIDEO_PATH = "C:/Users/Sergio/Desktop/Video/estacionamiento.mp4"
PICKLE_PATH = "C:/Users/Sergio/Desktop/pkl/rectangulos.pkl"

# =======================================================
# GENERAR RECTANGULOS
# =======================================================

def generar_rectangulo(p1, p2):      # Defino la función que crea un rectángulo ortogonal dado dos puntos opuestos
    x1, y1 = p1                      # Extrae coordenadas x,y del primer punto (esquina 1)
    x2, y2 = p2                      # Extrae coordenadas x,y del segundo punto (esquina opuesta)
    return np.array([                # Devuelve un array NumPy con los 4 vértices del rectángulo
        (x1, y1),                    # vértice superior-izquierda
        (x2, y1),                    # vértice superior-derecha
        (x2, y2),                    # vértice inferior-derecha
        (x1, y2)                     # vértice inferior-izquierda
    ], np.int32)                     # fuerza tipo entero de 32 bits para OpenCV

# =======================================================
# INDICE RECTANGULOS (Devuelve el índice del rectángulo cuyo centro está más cerca del punto dado)
# =======================================================

def indice_rectangulo(rects, point):  # Encuentra índice del rectángulo más cercano a un punto
    if not rects:                            # Si la lista de rectángulos está vacía...
        return -1                            # ...devuelve -1 indicando "no encontrado"

    px, py = point                           # Coordenadas x,y del punto de referencia
    min_dist = float("inf")                  # Inicializa distancia mínima con infinito
    idx = -1                                 # Índice del rectángulo más cercano (inicial -1)

    for i, rect in enumerate(rects):         # Recorre todos los rectángulos con su índice
        xs = [p[0] for p in rect]            # Lista de coordenadas x de los 4 vértices
        ys = [p[1] for p in rect]            # Lista de coordenadas y de los 4 vértices
        cx = sum(xs) / 4                     # Centro x del rectángulo (promedio de x)
        cy = sum(ys) / 4                     # Centro y del rectángulo (promedio de y)

        dist = math.hypot(px - cx, py - cy)  # Distancia entre punto y centro

        if dist < min_dist:                  # Si esta distancia es la nueva mínima...
            min_dist = dist                  # ...actualiza la mínima...
            idx = i                          # ...y guarda el índice del rectángulo

    return idx                               # Devuelve índice del rectángulo más cercano

# =======================================================
# RECUADRO
# =======================================================


def menu_instrucciones(img, lines, color=(255, 0, 0)): # Defino la función que dibuja un recuadro semi-transparente con texto de instrucciones
    overlay = img.copy()                               # Copia de la imagen para dibujar overlay
    alpha = 0.60                                       # Transparencia del fondo (0=transparente,1=opaco)
    height = 10 + 25 * len(lines) + 10                 # Calcula altura necesaria según número de líneas
    cv2.rectangle(overlay, (10, 10), (450, 10 + 25 * len(lines) + 10), (0, 0, 0), -1)  # Dibuja rectángulo negro lleno como fondo
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)  # Mezcla el overlay con la imagen original para transparencia

    y = 35                                             # Posición vertical inicial del primer renglón de texto
    for line in lines:                                 # Itera sobre cada línea de texto
        cv2.putText(img, line, (20, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, color, 2, cv2.LINE_AA)        # Escribe la línea en la imagen con fuente y grosor
        y += 25                                        # Avanza la coordenada vertical para la siguiente línea


# =======================================================
# MARCADO SOBRE FRAME Y MUESTRA DE INSTRUCCIONES
# =======================================================


def marcado_frame(primer_frame):                    # Defino la funcion para marcar rectángulos sobre el primer frame
    puntos = []                                    # Lista temporal para el/los puntos del rectángulo en construcción
    espacios = []                                  # Lista de rectángulos ya definidos (cada uno es un array de 4 puntos)
    vista = primer_frame.copy()                       # Copia del frame que se irá actualizando visualmente

    win = "Marcar Rectangulos"                     # Nombre de la ventana OpenCV
    cv2.namedWindow(win, cv2.WINDOW_AUTOSIZE)      # Crea ventana redimensionable automáticamente

    # Tiempo para mostrar instrucciones (6 segundos)
    show_instructions_until = time.time() + 6      # Marca el tiempo hasta el cual se mostrarán las instrucciones

    def redraw():                                                    # Defino la función para redibujar la vista desde el primer frame
        nonlocal vista
        vista = primer_frame.copy()                                  # Reinicia vista a la copia del primer frame

        # Dibujar rectángulos existentes
        for r in espacios:                                           # Para cada rectángulo ya creado...
            cv2.polylines(vista, [r], True, (0, 255, 255), 2)        # Dibuja el contorno con polilineas en amarillo
            overlay = vista.copy()                                   # Crea un overlay para rellenar el polígono
            cv2.fillPoly(overlay, [r], (0, 255, 255))                # Rellena el polígono en el overlay
            cv2.addWeighted(overlay, 0.2, vista, 0.8, 0, vista)      # Mezcla overlay semitransparente con vista

        # Punto inicial si estás creando uno nuevo
        if len(puntos) == 1:                                         # Si hay exactamente un punto en progreso...
            cv2.circle(vista, puntos[0], 4, (0, 255, 255), -1)       # Dibuja un círculo que marca el primer punto

        # Mostrar instrucciones solo durante 6 segundos
        if time.time() < show_instructions_until:  # Si aún no pasó el tiempo de mostrar instrucciones...
            menu_instrucciones(vista, [
                "INSTRUCCIONES:",
                " - 2 clics IZQUIERDOS para crear rectangulo",
                " - Clic DERECHO para borrar",
                " - ENTER para terminar, ESC para cancelar"
            ], color=(0, 165, 255))  # color naranja

    def click(ev, x, y, flags, param):           # Defino la funcion Callback de ratón que maneja clicks para crear/borrar rectángulos
        nonlocal puntos, espacios

        # Botón izquierdo → crear
        if ev == cv2.EVENT_LBUTTONDOWN:           # Si se presionó botón izquierdo...
            puntos.append((x, y))                 # Añade el punto a la lista temporal

            if len(puntos) == 2:                  # Si ya hay dos puntos (esquina opuesta)...
                rect = generar_rectangulo(puntos[0], puntos[1])  # Genera rectángulo ortogonal
                espacios.append(rect)            # Añade el rectángulo a la lista de espacios
                puntos = []                      # Resetea la lista de puntos
            redraw()                              # Redibuja la vista para reflejar cambios

        # Botón derecho ( borrar o cancelar en progreso)
        elif ev == cv2.EVENT_RBUTTONDOWN:         # Si se presionó botón derecho...
            if len(puntos) == 1:                  # Si había un punto en progreso, lo cancela
                puntos = []                       # Limpia el punto en progreso
                redraw()                          # Redibuja la vista
                return

            idx = indice_rectangulo(espacios, (x, y))  # Busca rectángulo más cercano al click
            if idx != -1:                         # Si encontró uno cercano...
                espacios.pop(idx)                # ...lo elimina de la lista

            redraw()                              # Redibuja la vista para actualizar borrado

    cv2.setMouseCallback(win, click)              # Asocia la función click al evento de ratón de la ventana

    redraw()                                      # Dibuja inicialmente la vista con instrucciones

    while True:                                   # Bucle principal de evento hasta que el usuario confirma o cancela
        cv2.imshow(win, vista)                      # Muestra la vista actualizada en la ventana
        k = cv2.waitKey(20) & 0xFF                  # Espera 20ms por tecla

        if k in (13, 10):  # ENTER                  # Si presiona ENTER (13 o 10) -> confirma y sale
            break
        if k == 27:  # ESC                         # Si presiona ESC (27) -> cancela, borra todo y sale
            espacios = []
            break

        redraw()                                   # Redibuja continuamente por si hay cambios por mouse

    cv2.destroyWindow(win)                         # Cierra la ventana de marcado

    with open(PICKLE_PATH, "wb") as f:
        pickle.dump(espacios, f)                   #Se sobreescribe el archivo .pkl



# =======================================================
# DIBUJO DE RECTANGULOS - MENU
# =======================================================


def dib_rectangulos(img, polys):                    # Defino la funcion que dibuja los rectangulos cargados sobre una imagen y muestra instrucciones
    vista = img.copy()                                # Copia la imagen para no alterar la original

    for poly in polys:                              # Para cada rectangulo en la lista
        arr = np.array(poly, np.int32)              # Asegura que el rectangulo sea un array de enteros
        cv2.polylines(vista, [arr], True, (0, 165, 255), 2)  # Dibuja el contorno en Naranja
        overlay = vista.copy()                        # Crea overlay para relleno semitransparente
        cv2.fillPoly(overlay, [arr], (0, 165, 255))   # Rellena el rectangulo en el overlay. Naranja
        cv2.addWeighted(overlay, 0.2, vista, 0.8, 0, vista)  # Mezcla el overlay con vis

    # Instrucciones SOLO por 3 segundos
    menu_instrucciones(vista, [
        "MENU",
        "m = marcar",
        "ENTER = usar archivo",
        "ESC = salir"
    ], color=(0, 165, 255))  # color naranja

    return vista                                      # Devuelve la imagen con rectangulos e instrucciones dibujadas

# =======================================================
# FUNCION PRINCIPAL
# =======================================================


def principal_main():                                 # Defino la funcion principal para video, frame y archivo pkl

    if not os.path.exists(VIDEO_PATH):               # Verifica que exista el video especificado
        print("Error: no se encuentra el video.")    # Mensaje de error si falta el video
        return False

    cap = cv2.VideoCapture(VIDEO_PATH)               # Abre el video
    ret, first = cap.read()                          # Lee el primer frame
    cap.release()                                    # Libera el capturador de video

    if not ret:                                      # Si no pudo leer el frame...
        print("No se pudo leer el primer frame.")    # ...informa y sale con False
        return False

    # Si existe archivo de rectángulos
    if os.path.exists(PICKLE_PATH):                  # Si el archivo pickle con rectangulos existe...
        try:
            with open(PICKLE_PATH, "rb") as f:       # Intenta cargarlo
                loaded = pickle.load(f)              # Carga la lista de rectangulos guardada
        except:
            loaded = []                              # Ante cualquier error, usa lista vacía

        vista = dib_rectangulos(first, loaded)           # Dibuja rectangulos sobre el primer frame

        win = "DETECCION"                            # Nombre de la ventana para mostrar opciones
        cv2.namedWindow(win, cv2.WINDOW_AUTOSIZE)    # Crea la ventana

        show_until = time.time() + 6  # mostrar instrucciones 6s  # Tiempo hasta el cual se ven las instrucciones

        while True:                                  # Bucle de interacción para aceptar, remarcar o salir
            tmp = vista.copy()                         # Copia de la imagen a mostrar

            if time.time() >= show_until:            # Si pasó el tiempo de mostrar instrucciones...
                # Borrar instrucciones después de 6 segundos
                tmp = first.copy()                   # Reemplaza tmp por la imagen original sin el bloque de instrucciones
                for poly in loaded:                  # Dibuja solo los contornos de los polígonos (sin relleno ni instrucciones)
                    arr = np.array(poly, np.int32)
                    cv2.polylines(tmp, [arr], True, (255, 0, 0), 2)

            cv2.imshow(win, tmp)                     # Muestra la imagen (con o sin instrucciones)
            k = cv2.waitKey(30) & 0xFF               # Espera por pulsación de tecla

            if k == ord('m'):                        # Si el usuario presiona 'm' -> entrar a modo de marcado manual
                cv2.destroyWindow(win)               # Cierra la ventana actual
                marcado_frame(first)                # Llama a la función que permite remarcar (y guardar)
                break                               # Sale del bucle para relanzar la comprobación al final

            elif k in (13, 10):  # ENTER               # ENTER -> usar el archivo cargado y retornar True
                cv2.destroyWindow(win)
                return True

            elif k == 27:  # ESC                       # ESC -> cancelar y retornar False
                cv2.destroyWindow(win)
                return False

    else:
        # No hay archivo: abrir modo manual directamente
        marcado_frame(first)                         # Si no existe archivo, entra directamente en marcado manual

    return os.path.exists(PICKLE_PATH)                # Al final retorna True si el archivo pickle ahora existe, False si no


# =======================================================
# PROCESADO Y CHEQUEO DE OCUPACIÓN
# =======================================================

# ---------------------------------
UMBRAL = 220   # valor inicial del umbral para decidir libre/ocupado

def on_trackbar(val):   # Callback para actualizar el umbral desde el trackbar
    global UMBRAL
    UMBRAL = val
# ----------------------------------


def chequeo_espacio(imgPro, poly):                             # Función que determina si un espacio está libre
                                                           #Devuelve True si está LIBRE, False si está OCUPADO.


    mask = np.zeros(imgPro.shape, dtype=np.uint8)          # Crea máscara negra del tamaño de la imagen procesada
    pts = np.array(poly, np.int32)                         # Convierte lista de vértices en array numpy
    cv2.fillPoly(mask, [pts], 255)                         # Rellena la zona con blanco en la máscara

    area = cv2.bitwise_and(imgPro, imgPro, mask=mask)      # Extrae solo los píxeles dentro del rectangulo
    cont = cv2.countNonZero(area)                         # Cuenta la cantidad de píxeles blancos

    return cont < UMBRAL                              #  si el cotador es menor al valor del umbral, el espacio se considera libre


# =======================================================
# DETECTOR PRINCIPAL
# =======================================================

def detect_main():                                         # Función principal del detector

    if not os.path.exists(PICKLE_PATH):                    # Verifica si existe archivo de rectangulos
        print("No hay polígonos guardados.")               # Mensaje si no existe
        return                                             # Termina función

    with open(PICKLE_PATH, "rb") as f:                     # Abre el archivo pickle
        rectangulos = pickle.load(f)                          # Carga la lista de rectangulos

    cap = cv2.VideoCapture(VIDEO_PATH)                     # Abre el video
    delay = 30                                             # Milisegundos entre frames

# =======================================================
# TRACKBAR
# =======================================================
    cv2.namedWindow("Detector de Estacionamiento")         # ventana principal
    cv2.createTrackbar("UMBRAL", "Detector de Estacionamiento", UMBRAL, 1000, on_trackbar)  # Trackbar Umbral 0-1000

    while True:                                            # Bucle principal del video

        ret, frame = cap.read()                            # Lee frame del video
        if not ret:                                        # Si no hay más frames
            break                                          # Sale del bucle

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)     # Convierte a escala de grises
        blur = cv2.GaussianBlur(gray, (3,3), 1)            # Suaviza imagen para reducir ruido

        thr = cv2.adaptiveThreshold(                       # Aplica threshold adaptativo
            blur, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            25, 16
        )

        median_img = cv2.medianBlur(thr, 5)                # Filtro mediana para limpiar
        kernel = np.ones((3,3), np.uint8)                  # Kernel para dilatación
        dil = cv2.dilate(median_img, kernel, iterations=1) # Dilata para resaltar zonas blancas

        cv2.imshow("Imagen procesada", dil)                # Muestra imagen procesada


        # ============================================================
        # RECORRER CADA RECTANGULO Y CONTAR PÍXELES BLANCOS
        # ============================================================

        cont_libre = 0                                     # Contador de espacios libres

        for rec in rectangulos:                              # Recorre cada rectangulo del archivo

            libre = chequeo_espacio(dil, rec)                 # Llama función que decide si está libre

            color = (0,255,0) if libre else (0,0,255)      # Color verde si esta libre, rojo si esta ocupado
            if libre:
                cont_libre += 1                            # Incrementa contador de libres

            arr = np.array(rec, np.int32)                 # Convierte vértices a array para OpenCV

            mask = np.zeros_like(dil)                      # Máscara en negro para este rectangulo
            cv2.fillPoly(mask, [arr], 255)                 # Rellena el rectangulo en blanco

            pix_area = cv2.countNonZero(                   # Cuenta píxeles blancos del rectangulo
                cv2.bitwise_and(dil, mask)
            )

         # Centro geométrico de un rectángulo mediante promedio de vértices
            xs = [p[0] for p in arr]
            ys = [p[1] for p in arr]
            cx = int(sum(xs) / 4)
            cy = int(sum(ys) / 4)



            cv2.putText(frame,                              # Escribe pixel count dentro del rectángulo
                        str(pix_area),
                        (cx - 20, cy),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.45,
                        (0, 165, 255),
                        2)

            cv2.polylines(frame, [arr], True, color, 2)     # Dibuja contorno del rectángulo

        # ============================================================
        # PANEL DE INFORMACIÓN INFERIOR
        # ============================================================

        total = len(rectangulos)                              # Total de espacios
        ocupados = total - cont_libre                     # Calcula ocupados

        font = cv2.FONT_HERSHEY_SIMPLEX                    # Tipo de letra
        scale = 0.8                                         # Tamaño texto
        thickness = 2                                       # Grosor texto

        text1 = "Estacionamientos disponibles: "            # Texto inicial
        text2 = str(cont_libre)                             # Número de libres
        text3 = f"/{total}  Ocupados: "                     # Texto mitad
        text4 = str(ocupados)                               # Número ocupados

        (w1, h1), _ = cv2.getTextSize(text1, font, scale, thickness)  # Ancho del texto1
        (w2, _), _ = cv2.getTextSize(text2, font, scale, thickness)   # Ancho texto2
        (w3, _), _ = cv2.getTextSize(text3, font, scale, thickness)   # Ancho texto3

        x = 10                                              # Margen izquierdo
        y = frame.shape[0] - 10                             # Ubicación en la parte inferior

        overlay = frame.copy()                              # Crea una copia para transparencia
        total_width = w1 + w2 + w3 + 50                     # Ancho total del panel

        cv2.rectangle(                                      # Dibuja rectángulo negro transparente
            overlay,
            (x-5, y-h1-10),
            (x+total_width, y+5),
            (0,0,0),
            -1
        )
        frame = cv2.addWeighted(overlay, 0.35, frame, 0.65, 0)  # Mezcla fondo y panel

        cv2.putText(frame, text1, (x, y), font, scale, (255,255,255), thickness)   # Muestra texto1
        x += w1                                              # Avanza posición
        cv2.putText(frame, text2, (x, y), font, scale, (0,255,0), thickness)       # Texto2 en verde
        x += w2 + 5
        cv2.putText(frame, text3, (x, y), font, scale, (255,255,255), thickness)   # Texto3
        x += w3
        cv2.putText(frame, text4, (x, y), font, scale, (0,0,255), thickness)       # Texto4 en rojo

        cv2.imshow("Detector de Estacionamiento", frame)     # Muestra resultado final

        if cv2.waitKey(delay) & 0xFF == 27:                  # Espera tecla ESC
            break                                            # Sale del loop

    cap.release()                                            # Libera el video
    cv2.destroyAllWindows()                                  # Cierra ventanas


# =============================
# FLUJO PRINCIPAL
# =============================
def main():                                                  # Función principal
    print("Iniciando herramienta de marcado / detector.")    # Mensaje inicial
    ok = principal_main()                                       # Ejecuta herramienta de marcado
    if not ok:                                               # Si no crearon rectangulos
        print("No se generaron rectangulos. Saliendo.")        # Mensaje informativo
        return                                               # Sale del programa

    print("Iniciando detector con los rectangulos guardados...")  # Mensaje
    detect_main()                                            # Llama al detector
    print("Detector finalizado.")                            # Mensaje final

if __name__ == "__main__":                                   # Punto de entrada
    main()                                                   # Ejecuta programa


Iniciando herramienta de marcado / detector.
Error: no se encuentra el video.
No se generaron rectangulos. Saliendo.
